In [213]:
import pandas as pd
import numpy as np

df_game_data_in = pd.read_csv("./clean_data/game_results.csv").drop(['Unnamed: 0'], axis=1)
df_season_results_in = pd.read_csv("./clean_data/season_results.csv").drop(['Unnamed: 0'], axis=1)

df_game_data = df_game_data_in
df_season_results = df_season_results_in

df_game_data has 12 columns. Attendance in NaN for 2020. Appears to be from future games or Covid games in 2020. Games that were walkoffs have W or L followed by wo in the win_loss_result column. Data types are messed up. 

In [214]:
df_game_data.head(-10)
df_game_data[df_game_data["Date"]== '2020-09-06'].head(5)

,Date,Team,H/A,Opponent,win_loss_result,Runs,Runs Allowed,Game Time,Day/Night,Attendance,Game Number,Year
40,2020-09-06,ARI,A,SFG,L,2.0,4.0,2:52,D,NaN,41,2020
99,2020-09-06,ATL,H,WSN,W,10.0,3.0,3:26,D,NaN,40,2020
159,2020-09-06,BAL,H,NYY,W,5.0,1.0,3:02,D,NaN,40,2020
221,2020-09-06,BOS,H,TOR,L,8.0,10.0,4:23,D,NaN,42,2020
280,2020-09-06,CHC,H,STL,L,3.0,7.0,3:17,N,NaN,41,2020


For this analysis I can focus only on games that have a result

In [215]:
df_game_data = df_game_data[df_game_data['win_loss_result'].notna()]


Need to resolve some data type issues so I can do math. Attendance has a comma 

In [216]:
avg_attendance = round(df_game_data['Attendance'].str.replace(",", "").dropna().astype('int').mean(),0) #will use to impute missing attendance numbers
print(f'average attendance: {avg_attendance}')

#converting attendance into number and setting nulls to the average
df_game_data.loc[:, 'Attendance'] = df_game_data['Attendance'].fillna(str(avg_attendance)).str.replace(",", "").astype('float').astype('int')

#2020 had legit zero attendance...setting backto zero
df_game_data.loc[:, 'Attendance'] = np.where(df_game_data['Date'] < '2021-01-01', 0, df_game_data['Attendance'])


average attendance: 26221.0


In [217]:
data_types = {
    'Date': 'datetime64[ns]',
    'Team': 'string',
    'H/A': 'string',
    'Opponent':'string',
    'win_loss_result':'string',
    'Runs': 'int',
    'Runs Allowed': 'int',
    'Game Time':'string',
    'Day/Night': 'string',
    'Attendance':'int',
    'Game Number': 'int',
    'Year':'int'
}

df_game_data = df_game_data.astype(data_types)
df_game_data.dtypes

Date               datetime64[ns]
Team               string[python]
H/A                string[python]
Opponent           string[python]
win_loss_result    string[python]
Runs                        int64
Runs Allowed                int64
Game Time          string[python]
Day/Night          string[python]
Attendance                  int64
Game Number                 int64
Year                        int64
dtype: object

The season results looks good. The GB column -- for teams that won, or are in first in their division.

In [33]:
df_season_results.head(10)

,Season,Team Name,Wins,Losses,Win%,GB,Team
0,2020,Tampa Bay Rays,40,20,0.667,--,TBR
1,2020,New York Yankees,33,27,0.550,7.0,NYY
2,2020,Toronto Blue Jays,32,28,0.533,8.0,TOR
3,2020,Baltimore Orioles,25,35,0.417,15.0,BAL
4,2020,Boston Red Sox,24,36,0.400,16.0,BOS
5,2020,Minnesota Twins,36,24,0.600,--,MIN
6,2020,Cleveland Indians,35,25,0.583,1.0,NaN
7,2020,Chicago White Sox,35,25,0.583,1.0,CHW
8,2020,Kansas City Royals,26,34,0.433,10.0,KCR
9,2020,Detroit Tigers,23,35,0.397,12.0,DET


In [35]:
#join game_results with season results. 
game_data['Previous Year'] = game_data['Year'] - 1

analysis_data = pd.merge(
    game_data[['Game Number', 'Date', 'Team', 'Opponent','Year', 'Previous Year']], 
    season_results[['Season', 'Team', 'Win%']], 
    how='left', 
    left_on=['Opponent', 'Previous Year'], 
    right_on=['Team', 'Season'],   
    ).drop(['Team_y'], axis=1)

analysis_data.head()


,Game Number,Date,Team_x,Opponent,Year,Previous Year,Season,Win%
0,1,2020-07-24,ARI,SDP,2020,2019,NaN,NaN
1,2,2020-07-25,ARI,SDP,2020,2019,NaN,NaN
2,3,2020-07-26,ARI,SDP,2020,2019,NaN,NaN
3,4,2020-07-27,ARI,SDP,2020,2019,NaN,NaN
4,5,2020-07-28,ARI,TEX,2020,2019,NaN,NaN


In [45]:
(analysis_data[(analysis_data['Date'] > '2021-01-01') & (analysis_data['Game Number'] <= 30)]
 .drop(['Previous Year', 'Season'], axis=1)
 .groupby(by = ['Team_x', 'Year'], group_keys=False)
 .agg(
     Mean = ('Win%', np.mean)
 )
 #.agg(Mean = ('Mean', np.max))
 )


Mean
Team_x Year          
ARI    2021  0.512500
       2022  0.493333
       2023  0.541667
       2024  0.478467
       2025  0.514067
...               ...
WSN    2021  0.532833
       2022  0.478000
       2023  0.501833
       2024  0.541296
       2025  0.507167

[150 rows x 1 columns]